In [ ]:
import os, re, time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException

download_dir = r"C:\Users\rando\Downloads" 
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 20)

driver.get("https://vahan.parivahan.gov.in/vahan4dashboard/vahan/vahan/view/reportview.xhtml")

# --- CSS selectors for dropdown items ---
state_items_css  = "#j_idt34_items li"
rto_items_css    = "#selectedRto_items li"
yaxis_items_css  = "#yaxisVar_items li"
xaxis_items_css  = "#xaxisVar_items li"
year_items_css   = "#selectedYear_items li"

# --- Helper for safe clicks ---
def safe_click(locator, retries=3, delay=1):
    for attempt in range(retries):
        try:
            el = wait.until(EC.element_to_be_clickable(locator))
            driver.execute_script("arguments[0].click();", el)
            return
        except StaleElementReferenceException:
            print("Stale element, retrying:", locator)
            time.sleep(delay)
    raise Exception(f"Could not click {locator} after {retries} retries")


# --- Select Month ---
def select_month(month_label, retries=3):
    for attempt in range(retries):
        try:
            # Always re-open the dropdown to get a fresh DOM
            safe_click((By.ID, "groupingTable:selectMonth_label"))
            time.sleep(0.4)

            # Re-locate the overlay items each time
            month_items = wait.until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#groupingTable\\:selectMonth_items li"))
            )

            # Find the desired month by text
            for m in month_items:
                if m.text.strip() == month_label:
                    driver.execute_script("arguments[0].click();", m)
                    return
        except StaleElementReferenceException:
            print(f"Stale element while selecting {month_label}, retrying...")
            time.sleep(0.5)
    raise Exception(f"Could not select month {month_label} after {retries} retries")


all_dataframes = []

# --- Open States dropdown once to get list ---
safe_click((By.ID, "j_idt34_label")) #"j_idt38_label"
time.sleep(0.5)
states = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, state_items_css)))

for i in range(23, 26):  #len(states) # skip "All States" #do haryana
    safe_click((By.ID, "j_idt34_label"))
    time.sleep(0.4)
    states = driver.find_elements(By.CSS_SELECTOR, state_items_css)

    state = states[i]
    print("\nSTATE:", state.text)
    driver.execute_script("arguments[0].click();", state)
    time.sleep(.5)

    # --- Wait for RTOs ---
    wait.until(lambda d: len(d.find_elements(By.CSS_SELECTOR, rto_items_css)) > 1)
    safe_click((By.ID, "selectedRto_label"))
    time.sleep(0.4)
    rtos = driver.find_elements(By.CSS_SELECTOR, rto_items_css)

    for r in rtos[1:]:   # skip "All Offices"
        print("RTO:", r.text)
        driver.execute_script("arguments[0].click();", r)
        time.sleep(.3)

        # --- Set Y-Axis ---
        safe_click((By.ID, "yaxisVar_label"))
        time.sleep(0.3)
        for opt in driver.find_elements(By.CSS_SELECTOR, yaxis_items_css):
            if "Vehicle Category" in opt.text:
                driver.execute_script("arguments[0].click();", opt)
                break

        # --- Set X-Axis = Month Wise ---
        safe_click((By.ID, "xaxisVar_label"))   # open dropdown
        time.sleep(0.4)

        xaxis_opts = driver.find_elements(By.CSS_SELECTOR, "#xaxisVar_items li")
        for opt in xaxis_opts:
            if "Vehicle Class" in opt.text or "Vehicle Class" in opt.text:
                driver.execute_script("arguments[0].click();", opt)
                break

        # --- Set Year ---
        safe_click((By.ID, "selectedYear_label"))
        time.sleep(0.4)
        for opt in driver.find_elements(By.CSS_SELECTOR, year_items_css):
            if "2023" in opt.text:
                driver.execute_script("arguments[0].click();", opt)
                break

        # --- Click Refresh ---
        safe_click((By.ID, "j_idt65"))
        time.sleep(.5)
        print("\tRefreshed for", state.text, "->", r.text)
        
        
        # --- Select Month ---

        months = ["JAN","FEB","MAR","APR","MAY","JUN","JUL","AUG","SEP","OCT","NOV","DEC"]
        for month in months:
            select_month(month)
            time.sleep(0.3)
            # --- Click Excel download ---
            safe_click((By.ID, "groupingTable:xls"))
            time.sleep(.6)

            # files = sorted([os.path.join(download_dir, f) for f in os.listdir(download_dir)], key=os.path.getmtime)
            # path = files[-1]

            # # --- Detect format ---
            # ext = os.path.splitext(path)[1].lower()

            # try:
            #     if ext in [".xls", ".xlsx"]:
            #         # Try Excel first
            #         try:
            #             df = pd.read_excel(path, engine="openpyxl")  # for .xlsx
            #         except Exception:
            #             # Fallback: treat as CSV if Excel parsing fails
            #             df = pd.read_csv(path)
            #     else:
            #         # Default to CSV
            #         df = pd.read_csv(path)

            #     print("Downloaded shape:", df.shape)


                # --- Clean content ---
                # content = df.iloc[2:, 1:]
                # category_class = df.columns[0]
                # rto_clean = re.sub(r"\s+", "", category_class, flags=re.IGNORECASE)
                # content["RTO"] = [rto_clean] * content.shape[0]
                # content["STATE"] = state.text

                # # --- Check empty/non-empty ---
                # if df.shape[1] > 8 and df.shape[0]>3:
                #     print("✅ Non-empty table for", state.text, "->", r.text)
                # else:
                #     print("⚠️ Empty table for", state.text, "->", r.text)

                # all_dataframes.append(content)
            
            # except Exception as e:
            #     print("❌ Failed to parse downloaded file:", e)

# if all_dataframes:
#     master_df = pd.concat(all_dataframes, ignore_index=True)
#     master_df.to_excel("vahan_master_class.xlsx", index=False)
#     print("Saved consolidated data to vahan_master_class.xlsx")
# else:
#     print("No data collected.")

driver.quit()

In [6]:
if all_dataframes:
    master_df = pd.concat(all_dataframes, ignore_index=True)
    master_df.to_excel("vahan_master_class.xlsx", index=False)
    print("Saved consolidated data to vahan_master.xlsx")

Saved consolidated data to vahan_master.xlsx


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

driver = webdriver.Chrome()
wait = WebDriverWait(driver, 20)

driver.get("https://vahan.parivahan.gov.in/vahan4dashboard/vahan/vahan/view/reportview.xhtml")

# --- Locators ---
state_label      = (By.ID, "j_idt38_label")
state_items_css  = "#j_idt38_items li"
rto_label        = (By.ID, "selectedRto_label")
rto_items_css    = "#selectedRto_items li"

yaxis_label      = (By.ID, "yaxisVar_label")
yaxis_items_css  = "#yaxisVar_items li"
xaxis_label      = (By.ID, "xaxisVar_label")
xaxis_items_css  = "#xaxisVar_items li"
year_label       = (By.ID, "selectedYear_label")
year_items_css   = "#selectedYear_items li"

refresh_btn      = (By.ID, "j_idt70")

# --- Open States dropdown once to get list ---
wait.until(EC.element_to_be_clickable(state_label)).click()
time.sleep(0.5)
states = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, state_items_css)))

for i in range(1, len(states)):   # skip "All States"
    # Re-open dropdown each loop
    wait.until(EC.element_to_be_clickable(state_label)).click()
    time.sleep(0.4)
    states = driver.find_elements(By.CSS_SELECTOR, state_items_css)

    state = states[i]
    print("\nSTATE:", state.text)
    driver.execute_script("arguments[0].click();", state)
    time.sleep(1.0)

    # --- Wait for RTOs to populate ---
    wait.until(lambda d: len(d.find_elements(By.CSS_SELECTOR, rto_items_css)) > 1)
    rtos = driver.find_elements(By.CSS_SELECTOR, rto_items_css)

    # Open RTO dropdown
    wait.until(EC.element_to_be_clickable(rto_label)).click()
    time.sleep(0.4)
    rtos = driver.find_elements(By.CSS_SELECTOR, rto_items_css)

    for r in rtos[1:]:   # skip "All Offices"
        print("RTO:", r.text)
        driver.execute_script("arguments[0].click();", r)
        time.sleep(1.0)

        # --- Set Y-Axis = Vehicle Category ---
        wait.until(EC.element_to_be_clickable(yaxis_label)).click()
        time.sleep(0.4)
        yaxis_opts = driver.find_elements(By.CSS_SELECTOR, yaxis_items_css)
        for opt in yaxis_opts:
            if "Vehicle Category" in opt.text:
                driver.execute_script("arguments[0].click();", opt)
                break

        # --- Set X-Axis = Month Wise ---
        wait.until(EC.element_to_be_clickable(xaxis_label)).click()
        time.sleep(0.4)
        xaxis_opts = driver.find_elements(By.CSS_SELECTOR, xaxis_items_css)
        for opt in xaxis_opts:
            if "Month Wise" in opt.text:
                driver.execute_script("arguments[0].click();", opt)
                break

        # --- Set Year = 2025 ---
        wait.until(EC.element_to_be_clickable(year_label)).click()
        time.sleep(0.4)
        year_opts = driver.find_elements(By.CSS_SELECTOR, year_items_css)
        for opt in year_opts:
            if "2025" in opt.text:
                driver.execute_script("arguments[0].click();", opt)
                break

        # --- Click Refresh ---
        btn = wait.until(EC.element_to_be_clickable(refresh_btn))
        driver.execute_script("arguments[0].click();", btn)

        # 
        
        time.sleep(3)
        print("   Refreshed for", state.text, "->", r.text)

time.sleep(2)
driver.quit()

In [5]:
%load_ext autoreload
%autoreload 2
import time, base64,ssl, os, re,json5, pprint,random , math, hashlib, inspect,requests
from selenium import webdriver# Controls the browser
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By                   # Locators (ID, CLASS_NAME, XPATH, etc.)
from selenium.webdriver.support.ui import WebDriverWait       # Waits for elements to appear
from selenium.webdriver.support import expected_conditions as EC  # Conditions like "visible", "clickable"
from selenium.webdriver.chrome.options import Options
from urllib.parse import urljoin
import pandas as pd
import numpy as np
from io import StringIO

amfii_path = r"https://www.amfiindia.com/otherdata/fund-performance"

In [ ]:
import requests
import pandas as pd
import time

def fetch_all_cities():
    base_url = "https://www.amfiindia.com/api/distributor-agent"
    throttle = 3
    all_cities = set()

    # First request to get total pages
    params = {
        "strOpt": "ALL",
        "city": "",
        "search": "",
        "page": 1,
        "pageSize": 100
    }

    response = requests.get(base_url, params=params, verify=False)
    response.raise_for_status()
    data = response.json()

    page_count = data["meta"]["pageCount"]
    print(f"Total pages: {page_count}")

    # Collect first page
    for rec in data.get("data", []):
        all_cities.add(rec.get("City"))

    # Loop through remaining pages
    for page in range(2, page_count + 1):
        params["page"] = page
        response = requests.get(base_url, params=params, verify=False)
        response.raise_for_status()
        page_data = response.json()
        for rec in page_data.get("data", []):
            all_cities.add(rec.get("City"))
        time.sleep(throttle)

    return sorted(all_cities)

if __name__ == "__main__":
    cities = fetch_all_cities()
    print(f"Total unique cities: {len(cities)}")
    
    with open("cities.csv", "w", encoding="utf-8") as f:
        for city in cities:
            f.write(city + "\n")
    print(cities)  # show first 50 as sample